In [2]:
import os
data_path = '/home/sldev1/Project/hyeongeun_test/data/FER/'
test_img_ = len(os.listdir(data_path+"val_set/images/"))
test_ano_ = len(os.listdir(data_path+"val_set/annotations/"))
train_img_ = len(os.listdir(data_path+"train_set/images/"))
train_ano_ = len(os.listdir(data_path+"train_set/annotations/"))


print(f"Test Dataset_Image 개수 : {test_img_}")
print(f"Test Dataset_Annotation 개수 : {test_ano_}")
print("**************\n\n***************")
print(f"Train Dataset_Image 개수 : {train_img_}")
print(f"Train Dataset_Annotation 개수 : {train_ano_}")

Test Dataset_Image 개수 : 3999
Test Dataset_Annotation 개수 : 15996
**************

***************
Train Dataset_Image 개수 : 287651
Train Dataset_Annotation 개수 : 1150604


----

# 22.11.01 (화)
### 느낀점 : 데이터 전처리 힘들게 완료
1. Custom Dataset 안에 init에는 경로만 들어가면 된다.   
2. getitem 인덱스는 하나씩 꺼내먹을 수 도 있고, dataloader에서 batch묶음으로 들어가기도 하는 존재다.

----
# 11.02 (수) 해야하는 것
1. 데이터 전처리 다했으니, 이제 학습하는 코드 짜기

In [257]:
from torch.utils.data import Dataset
import glob


def img_load(path, train=True):
    if train:
        img_path = path+'/train_set/images/'
    else:
        img_path = path+'/val_set/images/'

    jpg = glob.glob(img_path+'*.jpg')
    sort_jpg = sorted(jpg, key=lambda s: int(re.findall(r'\d+', s)[1]))
    return sort_jpg
    #print ("img_list_jpg: {}".format(img_list_jpg))
    #print ("img_list_jpg: {}".format(sort_img_list_jpg))


def label_load(path, train=True):
    if train:
        label_path = path+'/train_set/annotations/'
    else:
        label_path = path+'/val_set/annotations/'

    label = glob.glob(label_path+'*exp.npy')
    #label
    sort_label = sorted(label,key=lambda s: int(re.findall(r'\d+', s)[1]))
    return sort_label
    

class MyFER_Dataset(Dataset):
    
    def __init__(self, img_data, label_data, train=True, transform=None):
        self.img = img_load(img_data)
        self.label = label_load(label_data)
        self.transform = transform
    
    def __getitem__(self, idx):
        self.img = Image.open(self.img[idx]) 
        self.label = np.array(self.label[idx])
        sample =  self.img
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    

    def __len__(self):
        return len(self.img)

In [258]:
x = label_load(path = data_path, train=False)
x = np.array(x)
print(x.shape)
print(type(x))


(3999,)
<class 'numpy.ndarray'>


In [259]:
import torch
import re
import PIL
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as tr

path = "/home/sldev1/Project/hyeongeun_test/data/FER"

In [260]:
transform_train = tr.Compose([tr.ToTensor(),
                              tr.Normalize])
transform_test = tr.Compose([tr.ToTensor()])


train_dataset = MyFER_Dataset(img_data = path,
                              label_data = path,
                              train=True,
                              transform=transform_train)
test_dataset = MyFER_Dataset(img_data = path,
                              label_data = path,
                              train=False,
                              transform=transform_test)

In [261]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [263]:
test_dataset[0]

tensor([[[0.6667, 0.6902, 0.6863,  ..., 0.7451, 0.7490, 0.7529],
         [0.6941, 0.6863, 0.6627,  ..., 0.7412, 0.7490, 0.7529],
         [0.7098, 0.6627, 0.6235,  ..., 0.7412, 0.7490, 0.7529],
         ...,
         [0.0275, 0.0157, 0.0196,  ..., 0.5294, 0.5294, 0.5333],
         [0.0314, 0.0196, 0.0157,  ..., 0.5294, 0.5294, 0.5255],
         [0.0353, 0.0196, 0.0157,  ..., 0.5294, 0.5255, 0.5176]],

        [[0.5843, 0.6078, 0.6078,  ..., 0.6980, 0.7020, 0.7059],
         [0.6118, 0.6039, 0.5804,  ..., 0.6941, 0.7020, 0.7059],
         [0.6275, 0.5804, 0.5412,  ..., 0.6941, 0.7020, 0.7059],
         ...,
         [0.0392, 0.0275, 0.0314,  ..., 0.5412, 0.5412, 0.5451],
         [0.0431, 0.0314, 0.0275,  ..., 0.5412, 0.5412, 0.5373],
         [0.0471, 0.0314, 0.0275,  ..., 0.5412, 0.5373, 0.5294]],

        [[0.3608, 0.3843, 0.3765,  ..., 0.4941, 0.4980, 0.5020],
         [0.3882, 0.3804, 0.3569,  ..., 0.4902, 0.4980, 0.5020],
         [0.4118, 0.3569, 0.3176,  ..., 0.4902, 0.4980, 0.